In [1]:
import torch
from torch.utils.data import DataLoader
from dataset.dataset import get_cdiscount_dataset
from model.model import assemble_model, assemble_model_with_classifier
from model.utils import freeze_layers
from trainer.trainer import get_trainer

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# redirect print to file
# import sys
# sys.stdout = open("PyTorch-resnet34-log.txt", "w")

In [2]:
# configuration
config = {
    'train_batch_size': 512, 'val_batch_size': 512,
    'arch': 'resnet50', 'pretrained': True,
    'optimizer': 'Adam', 'learning_rate': 1e-3, 'decay_lr_freq': 2e4, 'weight_decay': 1e-5,
    'resume': None,
    'start_epoch': 0, 'epochs': 10,
    'print_freq': 10, 'validate_freq': 2e4, 'save_freq': 1e3,
    'best_val_prec1': 0
}

In [3]:
import torchvision.models as models

# get dataset
print('getting dataset...')
train_dataset = get_cdiscount_dataset(offsets_csv="train_offsets.csv",
                                      images_csv="train_images.csv",
                                      bson_file_path="/mnt/data/cdiscount/train.bson",
                                      with_label=True,
                                      resize=224)
val_dataset = get_cdiscount_dataset(offsets_csv="train_offsets.csv",
                                    images_csv="val_images.csv",
                                    bson_file_path="/mnt/data/cdiscount/train.bson",
                                    with_label=True,
                                    resize=224)

# get data loader
print('getting data loader...')
train_dataloader = DataLoader(train_dataset, batch_size=config['train_batch_size'], shuffle=True, num_workers=1)
val_dataloader = DataLoader(val_dataset, batch_size=config['val_batch_size'], shuffle=True, num_workers=1)

# define model
print("=> using pre-trained model '{}'".format(config['arch']))
model = models.__dict__[config['arch']](pretrained=config['pretrained'])

# model = torch.nn.Sequential(*list(model.children())[:-1])
# model.add_module('classifier', torch.nn.Linear(in_features=2048, out_features=5270))
# model = torch.nn.DataParallel(model).cuda()
# print(model)

freeze_layers(model, 6)

classifier_layer = [
    torch.nn.Linear(in_features=2048, out_features=5270)
]

# classifier_layer = [
#     torch.nn.Linear(in_features=2048, out_features=5270),
# ]

classifier = torch.nn.Sequential(*classifier_layer)

model = assemble_model_with_classifier(model, -1, classifier)
model = torch.nn.DataParallel(model).cuda()
print(model)

getting dataset...
getting data loader...
=> using pre-trained model 'resnet50'
0 freezing Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
1 freezing BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
2 freezing ReLU (inplace)
3 freezing MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
4 freezing Sequential (
  (0): Bottleneck (
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (relu): ReLU (inplace)
    (downsample): Sequential (
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momen

In [ ]:
# define loss function (criterion) and optimizer
criterion = torch.nn.CrossEntropyLoss().cuda()

# get trainer
Trainer = get_trainer(train_dataloader, val_dataloader, model, criterion, config)

# Run!
Trainer.run()

start training
Epoch: [0][0/19340]	Time 41.062 (41.062)	Data 5.523 (5.523)	Loss 8.6404 (8.6404)	Prec@1 0.000 (0.000)	Prec@5 0.195 (0.195)
Epoch: [0][10/19340]	Time 4.427 (7.690)	Data 0.014 (0.516)	Loss 6.4328 (7.1239)	Prec@1 15.039 (9.162)	Prec@5 28.320 (18.164)
Epoch: [0][20/19340]	Time 4.429 (6.135)	Data 0.016 (0.277)	Loss 5.9433 (6.6811)	Prec@1 16.797 (11.812)	Prec@5 29.492 (22.535)
Epoch: [0][30/19340]	Time 4.394 (5.583)	Data 0.378 (0.243)	Loss 5.6552 (6.4075)	Prec@1 18.945 (13.552)	Prec@5 35.742 (25.662)
Epoch: [0][40/19340]	Time 4.467 (5.344)	Data 0.939 (0.427)	Loss 5.2408 (6.1736)	Prec@1 20.508 (15.010)	Prec@5 37.109 (27.887)
Epoch: [0][50/19340]	Time 4.526 (5.188)	Data 0.984 (0.540)	Loss 5.1994 (6.0076)	Prec@1 18.555 (15.993)	Prec@5 37.500 (29.615)
Epoch: [0][60/19340]	Time 4.616 (5.077)	Data 1.074 (0.607)	Loss 4.8425 (5.8469)	Prec@1 25.586 (17.063)	Prec@5 41.992 (31.340)
Epoch: [0][70/19340]	Time 4.494 (4.999)	Data 0.881 (0.654)	Loss 4.7790 (5.7137)	Prec@1 25.000 (17.960)	Prec

Epoch: [0][650/19340]	Time 4.477 (4.612)	Data 0.875 (0.915)	Loss 3.4214 (3.9430)	Prec@1 38.477 (33.622)	Prec@5 57.617 (52.317)
Epoch: [0][660/19340]	Time 4.549 (4.610)	Data 0.986 (0.914)	Loss 3.1251 (3.9318)	Prec@1 42.969 (33.753)	Prec@5 63.672 (52.451)
Epoch: [0][670/19340]	Time 4.498 (4.609)	Data 0.946 (0.914)	Loss 3.2152 (3.9211)	Prec@1 41.406 (33.871)	Prec@5 60.742 (52.578)
Epoch: [0][680/19340]	Time 4.587 (4.608)	Data 1.021 (0.915)	Loss 3.1367 (3.9105)	Prec@1 43.359 (33.984)	Prec@5 61.914 (52.707)
Epoch: [0][690/19340]	Time 4.709 (4.609)	Data 1.128 (0.917)	Loss 3.3791 (3.9003)	Prec@1 39.062 (34.089)	Prec@5 58.789 (52.833)
Epoch: [0][700/19340]	Time 4.495 (4.608)	Data 0.937 (0.917)	Loss 3.4594 (3.8907)	Prec@1 39.062 (34.200)	Prec@5 56.641 (52.945)
Epoch: [0][710/19340]	Time 4.598 (4.607)	Data 1.073 (0.918)	Loss 3.1534 (3.8822)	Prec@1 42.383 (34.304)	Prec@5 63.867 (53.048)
Epoch: [0][720/19340]	Time 4.661 (4.608)	Data 1.086 (0.920)	Loss 3.5134 (3.8731)	Prec@1 37.500 (34.397)	Prec@5 

Epoch: [0][1300/19340]	Time 4.510 (4.575)	Data 0.019 (0.786)	Loss 2.8938 (3.5063)	Prec@1 44.922 (38.575)	Prec@5 64.844 (57.708)
Epoch: [0][1310/19340]	Time 4.495 (4.574)	Data 0.015 (0.781)	Loss 2.8899 (3.5012)	Prec@1 44.531 (38.637)	Prec@5 66.992 (57.776)
Epoch: [0][1320/19340]	Time 4.540 (4.573)	Data 0.020 (0.775)	Loss 3.0045 (3.4972)	Prec@1 45.312 (38.686)	Prec@5 64.844 (57.830)
Epoch: [0][1330/19340]	Time 4.460 (4.573)	Data 0.021 (0.769)	Loss 2.7988 (3.4926)	Prec@1 46.875 (38.737)	Prec@5 66.992 (57.888)
Epoch: [0][1340/19340]	Time 4.482 (4.572)	Data 0.018 (0.764)	Loss 2.8467 (3.4886)	Prec@1 44.922 (38.784)	Prec@5 65.820 (57.938)
Epoch: [0][1350/19340]	Time 4.497 (4.571)	Data 0.017 (0.758)	Loss 2.9552 (3.4844)	Prec@1 45.703 (38.834)	Prec@5 65.430 (57.990)
Epoch: [0][1360/19340]	Time 4.521 (4.571)	Data 0.017 (0.753)	Loss 3.0637 (3.4807)	Prec@1 44.727 (38.878)	Prec@5 64.062 (58.035)
Epoch: [0][1370/19340]	Time 4.515 (4.570)	Data 0.018 (0.748)	Loss 2.8128 (3.4765)	Prec@1 48.047 (38.929)

Epoch: [0][1950/19340]	Time 4.500 (4.553)	Data 0.800 (0.688)	Loss 2.6953 (3.2774)	Prec@1 50.195 (41.349)	Prec@5 68.555 (60.618)
Epoch: [0][1960/19340]	Time 4.544 (4.553)	Data 0.983 (0.688)	Loss 2.5854 (3.2747)	Prec@1 52.930 (41.385)	Prec@5 69.531 (60.651)
Epoch: [0][1970/19340]	Time 4.518 (4.553)	Data 0.968 (0.690)	Loss 2.7652 (3.2722)	Prec@1 47.461 (41.414)	Prec@5 66.602 (60.682)
Epoch: [0][1980/19340]	Time 4.531 (4.553)	Data 0.976 (0.691)	Loss 2.8335 (3.2692)	Prec@1 50.586 (41.452)	Prec@5 66.602 (60.720)
Epoch: [0][1990/19340]	Time 4.510 (4.553)	Data 0.915 (0.693)	Loss 2.5648 (3.2665)	Prec@1 50.000 (41.487)	Prec@5 69.336 (60.754)
Epoch: [0][2000/19340]	Time 4.470 (4.553)	Data 0.118 (0.692)	Loss 2.7595 (3.2637)	Prec@1 44.531 (41.514)	Prec@5 67.578 (60.790)
Epoch: [0][2010/19340]	Time 4.503 (4.553)	Data 0.379 (0.690)	Loss 2.8634 (3.2612)	Prec@1 45.703 (41.543)	Prec@5 66.797 (60.821)
Epoch: [0][2020/19340]	Time 4.590 (4.553)	Data 1.064 (0.691)	Loss 2.8097 (3.2586)	Prec@1 49.219 (41.575)

Epoch: [0][2600/19340]	Time 4.478 (4.551)	Data 0.900 (0.714)	Loss 2.4018 (3.1266)	Prec@1 52.930 (43.243)	Prec@5 71.289 (62.570)
Epoch: [0][2610/19340]	Time 4.477 (4.551)	Data 0.661 (0.715)	Loss 2.7608 (3.1247)	Prec@1 45.898 (43.267)	Prec@5 68.164 (62.596)
Epoch: [0][2620/19340]	Time 4.500 (4.550)	Data 0.029 (0.713)	Loss 2.6128 (3.1229)	Prec@1 50.195 (43.291)	Prec@5 69.141 (62.618)
Epoch: [0][2630/19340]	Time 4.694 (4.550)	Data 1.123 (0.713)	Loss 2.5123 (3.1210)	Prec@1 51.367 (43.317)	Prec@5 69.922 (62.642)
Epoch: [0][2640/19340]	Time 4.519 (4.550)	Data 0.924 (0.714)	Loss 2.9831 (3.1194)	Prec@1 45.703 (43.341)	Prec@5 65.039 (62.664)
Epoch: [0][2650/19340]	Time 4.527 (4.551)	Data 0.924 (0.715)	Loss 2.6560 (3.1174)	Prec@1 48.242 (43.366)	Prec@5 67.969 (62.690)
Epoch: [0][2660/19340]	Time 4.614 (4.551)	Data 1.057 (0.716)	Loss 2.4417 (3.1156)	Prec@1 50.000 (43.389)	Prec@5 70.703 (62.712)
Epoch: [0][2670/19340]	Time 4.624 (4.551)	Data 1.098 (0.717)	Loss 2.6114 (3.1136)	Prec@1 47.656 (43.414)

...

Epoch: [0][7150/19340]	Time 4.462 (4.532)	Data 0.040 (0.615)	Loss 2.4762 (2.6762)	Prec@1 50.781 (49.144)	Prec@5 72.852 (68.532)
Epoch: [0][7160/19340]	Time 4.489 (4.532)	Data 0.034 (0.614)	Loss 2.2155 (2.6757)	Prec@1 56.250 (49.151)	Prec@5 76.367 (68.540)
Epoch: [0][7170/19340]	Time 4.460 (4.532)	Data 0.041 (0.613)	Loss 2.1888 (2.6751)	Prec@1 57.031 (49.160)	Prec@5 74.609 (68.548)
Epoch: [0][7180/19340]	Time 4.472 (4.532)	Data 0.034 (0.612)	Loss 2.4055 (2.6745)	Prec@1 51.758 (49.166)	Prec@5 72.656 (68.554)
Epoch: [0][7190/19340]	Time 4.480 (4.532)	Data 0.034 (0.612)	Loss 2.2775 (2.6739)	Prec@1 53.906 (49.175)	Prec@5 75.586 (68.562)
Epoch: [0][7200/19340]	Time 4.498 (4.532)	Data 0.044 (0.611)	Loss 2.1863 (2.6734)	Prec@1 57.422 (49.182)	Prec@5 75.195 (68.569)
Epoch: [0][7210/19340]	Time 4.481 (4.532)	Data 0.043 (0.610)	Loss 2.3511 (2.6730)	Prec@1 50.586 (49.187)	Prec@5 72.852 (68.575)
Epoch: [0][7220/19340]	Time 4.489 (4.532)	Data 0.037 (0.609)	Loss 2.1542 (2.6724)	Prec@1 53.320 (49.195)

Epoch: [0][7800/19340]	Time 4.454 (4.529)	Data 0.071 (0.587)	Loss 2.3223 (2.6428)	Prec@1 54.297 (49.591)	Prec@5 75.000 (68.985)
Epoch: [0][7810/19340]	Time 4.513 (4.529)	Data 0.069 (0.587)	Loss 2.1520 (2.6423)	Prec@1 56.641 (49.597)	Prec@5 76.367 (68.992)
Epoch: [0][7820/19340]	Time 4.519 (4.529)	Data 0.057 (0.586)	Loss 2.2880 (2.6418)	Prec@1 54.297 (49.604)	Prec@5 73.438 (68.998)
Epoch: [0][7830/19340]	Time 4.488 (4.529)	Data 0.067 (0.585)	Loss 2.2709 (2.6413)	Prec@1 53.906 (49.612)	Prec@5 74.805 (69.007)
Epoch: [0][7840/19340]	Time 4.509 (4.529)	Data 0.057 (0.585)	Loss 2.4125 (2.6408)	Prec@1 53.906 (49.618)	Prec@5 73.242 (69.012)
Epoch: [0][7850/19340]	Time 4.461 (4.529)	Data 0.068 (0.584)	Loss 2.3541 (2.6403)	Prec@1 54.102 (49.625)	Prec@5 74.414 (69.019)
Epoch: [0][7860/19340]	Time 4.471 (4.529)	Data 0.274 (0.584)	Loss 2.1699 (2.6398)	Prec@1 55.078 (49.632)	Prec@5 75.781 (69.027)
Epoch: [0][7870/19340]	Time 4.541 (4.529)	Data 0.963 (0.584)	Loss 2.5109 (2.6393)	Prec@1 49.609 (49.639)

Epoch: [0][8450/19340]	Time 4.528 (4.529)	Data 0.950 (0.585)	Loss 2.4690 (2.6115)	Prec@1 52.344 (50.015)	Prec@5 72.852 (69.414)
Epoch: [0][8460/19340]	Time 4.591 (4.529)	Data 1.010 (0.586)	Loss 2.3921 (2.6111)	Prec@1 53.125 (50.021)	Prec@5 72.070 (69.420)
Epoch: [0][8470/19340]	Time 4.499 (4.529)	Data 0.830 (0.586)	Loss 2.2133 (2.6105)	Prec@1 58.203 (50.029)	Prec@5 75.586 (69.427)
Epoch: [0][8480/19340]	Time 4.473 (4.529)	Data 0.451 (0.586)	Loss 2.2712 (2.6101)	Prec@1 52.930 (50.035)	Prec@5 72.852 (69.434)
Epoch: [0][8490/19340]	Time 4.496 (4.529)	Data 0.330 (0.586)	Loss 2.2156 (2.6096)	Prec@1 54.492 (50.041)	Prec@5 74.805 (69.439)
Epoch: [0][8500/19340]	Time 4.479 (4.529)	Data 0.386 (0.586)	Loss 2.0304 (2.6092)	Prec@1 60.938 (50.049)	Prec@5 77.734 (69.445)
Epoch: [0][8510/19340]	Time 4.470 (4.529)	Data 0.736 (0.586)	Loss 2.1797 (2.6087)	Prec@1 55.859 (50.055)	Prec@5 75.000 (69.451)
Epoch: [0][8520/19340]	Time 4.475 (4.529)	Data 0.795 (0.586)	Loss 2.1595 (2.6082)	Prec@1 55.273 (50.061)

Epoch: [0][9100/19340]	Time 4.472 (4.527)	Data 0.233 (0.578)	Loss 2.3228 (2.5833)	Prec@1 55.078 (50.397)	Prec@5 71.875 (69.798)
Epoch: [0][9110/19340]	Time 4.496 (4.527)	Data 0.084 (0.577)	Loss 2.2422 (2.5829)	Prec@1 53.516 (50.403)	Prec@5 75.586 (69.804)
Epoch: [0][9120/19340]	Time 4.490 (4.527)	Data 0.085 (0.577)	Loss 2.1570 (2.5824)	Prec@1 58.008 (50.409)	Prec@5 74.414 (69.809)
Epoch: [0][9130/19340]	Time 4.490 (4.527)	Data 0.103 (0.576)	Loss 2.2426 (2.5821)	Prec@1 58.008 (50.415)	Prec@5 74.023 (69.814)
Epoch: [0][9140/19340]	Time 4.461 (4.527)	Data 0.068 (0.576)	Loss 2.2049 (2.5816)	Prec@1 55.469 (50.421)	Prec@5 74.414 (69.820)
Epoch: [0][9150/19340]	Time 4.508 (4.527)	Data 0.073 (0.575)	Loss 2.4750 (2.5813)	Prec@1 50.391 (50.426)	Prec@5 71.484 (69.825)
Epoch: [0][9160/19340]	Time 4.469 (4.527)	Data 0.065 (0.574)	Loss 2.3395 (2.5809)	Prec@1 52.148 (50.431)	Prec@5 71.289 (69.831)
Epoch: [0][9170/19340]	Time 4.487 (4.527)	Data 0.069 (0.574)	Loss 2.0420 (2.5805)	Prec@1 57.617 (50.437)

Epoch: [0][9750/19340]	Time 4.490 (4.526)	Data 0.076 (0.572)	Loss 2.1494 (2.5580)	Prec@1 56.836 (50.749)	Prec@5 74.609 (70.143)
Epoch: [0][9760/19340]	Time 4.458 (4.526)	Data 0.089 (0.572)	Loss 1.9514 (2.5576)	Prec@1 61.719 (50.754)	Prec@5 77.539 (70.149)
Epoch: [0][9770/19340]	Time 4.472 (4.526)	Data 0.078 (0.571)	Loss 2.1304 (2.5572)	Prec@1 56.641 (50.759)	Prec@5 75.195 (70.153)
Epoch: [0][9780/19340]	Time 4.488 (4.526)	Data 0.076 (0.571)	Loss 2.1523 (2.5568)	Prec@1 57.031 (50.765)	Prec@5 76.172 (70.159)
Epoch: [0][9790/19340]	Time 4.494 (4.526)	Data 0.070 (0.570)	Loss 2.3252 (2.5564)	Prec@1 53.125 (50.770)	Prec@5 73.633 (70.164)
Epoch: [0][9800/19340]	Time 4.455 (4.526)	Data 0.397 (0.570)	Loss 2.1488 (2.5561)	Prec@1 57.422 (50.775)	Prec@5 75.977 (70.170)
Epoch: [0][9810/19340]	Time 4.504 (4.526)	Data 0.873 (0.570)	Loss 2.0660 (2.5557)	Prec@1 58.008 (50.781)	Prec@5 78.516 (70.176)
Epoch: [0][9820/19340]	Time 4.527 (4.526)	Data 0.936 (0.570)	Loss 2.1171 (2.5553)	Prec@1 57.227 (50.786)